<a href="https://colab.research.google.com/github/brunoodon/sprintIII_datascience/blob/main/MVP_SPRINT3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import pandas as pd
import requests
import numpy as np
import csv
import json
import datetime
import time
import statistics
#url do dataset de ativos da empresa
url_inventory = 'https://raw.githubusercontent.com/brunoodon/mvp-sprint1/main/assets.csv'
#lendo e dando 'print' no dataset para análise dos campos e registros
inventory = pd.read_csv(url_inventory)
display(inventory)

,application,ip,hostname,dep
0,Exchange Server,172.16.0.20,mail.fakecomp.com,TI
1,iOS,10.0.0.200,fake-cellphone,Vendas
2,Windows,10.0.0.150,desktop01,Vendas
3,Chromium,10.0.0.150,desktop01,Vendas
4,Firefox,10.0.0.150,desktop01,Vendas
...,...,...,...,...
58,Pulse Connect Secure,10.30.2.1,vpn-srv,TI
59,Pulse Connect Secure,10.30.2.1,vpn-srv,TI
60,Pulse Connect Secure,10.30.2.1,vpn-srv,TI
61,Pulse Connect Secure,10.30.2.1,vpn-srv,TI


In [5]:
#url do dataset de vulnerabilidades recentemente exploradas
url_cve = 'https://www.cisa.gov/sites/default/files/csv/known_exploited_vulnerabilities.csv'
#lendo e dando 'print' no dataset para análise dos campos e registros
exploited_cve = pd.read_csv(url_cve)
display(exploited_cve)


,cveID,vendorProject,product,vulnerabilityName,dateAdded,shortDescription,requiredAction,dueDate,notes
0,CVE-2021-27104,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,NaN
1,CVE-2021-27102,Accellion,FTA,Accellion FTA OS Command Injection Vulnerability,2021-11-03,Accellion FTA contains an OS command injection...,Apply updates per vendor instructions.,2021-11-17,NaN
2,CVE-2021-27101,Accellion,FTA,Accellion FTA SQL Injection Vulnerability,2021-11-03,Accellion FTA contains a SQL injection vulnera...,Apply updates per vendor instructions.,2021-11-17,NaN
3,CVE-2021-27103,Accellion,FTA,Accellion FTA Server-Side Request Forgery (SSR...,2021-11-03,Accellion FTA contains a server-side request f...,Apply updates per vendor instructions.,2021-11-17,NaN
4,CVE-2021-21017,Adobe,Acrobat and Reader,Adobe Acrobat and Reader Heap-based Buffer Ove...,2021-11-03,Acrobat Acrobat and Reader contain a heap-base...,Apply updates per vendor instructions.,2021-11-17,NaN
...,...,...,...,...,...,...,...,...,...
1003,CVE-2022-31461,Owl Labs,Meeting Owl,Owl Labs Meeting Owl Missing Authentication fo...,2023-09-18,Owl Labs Meeting Owl contains a missing authen...,Apply mitigations per vendor instructions or d...,2023-10-16,This CVE is currently under review status. The...
1004,CVE-2022-31462,Owl Labs,Meeting Owl,Owl Labs Meeting Owl Use of Hard-coded Credent...,2023-09-18,Owl Labs Meeting Owl contains a use of hard-co...,Apply mitigations per vendor instructions or d...,2023-10-16,This CVE is currently under review status. The...
1005,CVE-2022-31463,Owl Labs,Meeting Owl,Owl Labs Meeting Owl Improper Authentication V...,2023-09-18,Owl Labs Meeting Owl contains an improper auth...,Apply mitigations per vendor instructions or d...,2023-10-16,This CVE is currently under review status. The...
1006,CVE-2023-28434,MinIO,MinIO,MinIO Security Feature Bypass Vulnerability,2023-09-19,MinIO contains a security feature bypass vulne...,Apply mitigations per vendor instructions or d...,2023-10-10,https://github.com/minio/minio/security/adviso...


In [ ]:
from pandas.io.parsers import python_parser
#percorrendo cada linha do dataset de inventário de ativos do cliente
for a in range(len(inventory)):
  application=inventory.values[a][0]
  ip=inventory.values[a][1]
  hostname=inventory.values[a][2]
  dep=inventory.values[a][3]
  #criando uma variável com a data atual, a ser atribuída como data do alerta
  today=str(datetime.date.today())
  #fazendo a busca dos registros do dataset de vulnerabilidades exploradas onde o campo 'Product' contém o nome da aplicação descrita em cada linha de inventário
  search = exploited_cve.query('product.str.contains("'+application+'") == True', engine="python")
  #percorrendo cada resultado dessa busca e atribuíndo os valores às variáveis
  for b in range(len(search)):
      cve=search.values[b][0]
      vendor=search.values[b][1]
      product=search.values[b][2]
      vulnerability_name=search.values[b][3]
      date_added=search.values[b][4]
      short_desc=search.values[b][5]
      due_date=search.values[b][6]
      notes=search.values[b][7]
      #fazendo a consulta do CVE no banco do NVD, para enriquecer o dataset
      url_nvd = 'https://services.nvd.nist.gov/rest/json/cves/2.0?cveId='+cve+''
      r_nvd = requests.get(url_nvd)
      try:
        #lendo o json com o resultado e atribuindo cada resultado de campo às varáveis severity_score e vector_string
        #obs: foi preciso estabelecer 2 possíveis estruturas, uma para cada versão da métrica de severity score (V2 ou V3.1)
        json_nvd = json.loads(r_nvd.text)
        for c in json_nvd['vulnerabilities']:
          try:
            for d in c['cve']['metrics']['cvssMetricV31']:
              vector_string = d['cvssData']['vectorString']
              #convertendo o campo de severity_score em float
              severity_score = float(d['cvssData']['baseScore'])
          except:
            for d in c['cve']['metrics']['cvssMetricV2']:
              vector_string = d['cvssData']['vectorString']
              severity_score = float(d['cvssData']['baseScore'])
          #foi necessário um temporizador porque a API do NVD tem limitação de requests por minuto
          time.sleep(8)
        #os dados correlacionados são inseridos como documentos JSON no índice alerts, no Elastisearch. Este é o dataset que será analisado e tratado
        es_url = 'http://mvp-puc.brunoodon.com.br:9200/vulnerability-alerts/_doc'
        es_header = {'Content-Type': 'application/json'}
        es_json = {
        'Vulnerability': cve,
        'VulnerabilityName': vulnerability_name,
        'Description': short_desc,
        'IP': ip,
        'Vendor': vendor,
        'Hostname': hostname,
        'Application': application,
        'ExploitationVector': vector_string,
        'SeverityScore': float(severity_score),
        'AlertDate': today,
        'Department': dep
        }
        r_es = requests.post(es_url, headers=es_header, verify=False, json=es_json)
        print(cve, vulnerability_name, short_desc, ip, vendor, hostname, application, vector_string, severity_score, today, dep)
      except:
        print('Nenhuma vulnerabilidade encontrada no ativo')


CVE-2020-17144 Microsoft Exchange Server Remote Code Execution Vulnerability Microsoft Exchange Server improperly validates cmdlet arguments which allow an attacker to perform remote code execution. 172.16.0.20 Microsoft mail.fakecomp.com Exchange Server CVSS:3.1/AV:N/AC:L/PR:H/UI:R/S:C/C:H/I:H/A:H 8.4 2023-09-25 TI
CVE-2021-34523 Microsoft Exchange Server Privilege Escalation Vulnerability Microsoft Exchange Server contains an unspecified vulnerability that allows for privilege escalation. 172.16.0.20 Microsoft mail.fakecomp.com Exchange Server CVSS:3.1/AV:N/AC:L/PR:N/UI:N/S:U/C:H/I:H/A:H 9.8 2023-09-25 TI
CVE-2020-0688 Microsoft Exchange Server Validation Key Remote Code Execution Vulnerability Microsoft Exchange Server Validation Key fails to properly create unique keys at install time, allowing for remote code execution. 172.16.0.20 Microsoft mail.fakecomp.com Exchange Server CVSS:3.1/AV:N/AC:L/PR:L/UI:N/S:U/C:H/I:H/A:H 8.8 2023-09-25 TI
CVE-2021-34473 Microsoft Exchange Server Rem